In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import os

In [3]:
# Konfigurasi direktori dan parameter pelatihan
train_dir = 'Validation'
validation_dir = 'Validation'
train_female_dir = os.path.join(train_dir, 'female')
train_male_dir = os.path.join(train_dir, 'male')
validation_female_dir = os.path.join(validation_dir, 'female')
validation_male_dir = os.path.join(validation_dir, 'male')

num_train_female = len(os.listdir(train_female_dir))
num_train_male = len(os.listdir(train_male_dir))
num_validation_female = len(os.listdir(validation_female_dir))
num_validation_male = len(os.listdir(validation_male_dir))

total_train = num_train_female + num_train_male
total_validation = num_validation_female + num_validation_male

batch_size = 32
epochs = 10

In [4]:
# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=(150, 150),
                                                              batch_size=batch_size,
                                                              class_mode='binary')


Found 11649 images belonging to 2 classes.
Found 11649 images belonging to 2 classes.


In [5]:
# Membuat model CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

c:\Users\alfia\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Kompilasi model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
# Melatih model
history = model.fit(train_generator,
                    steps_per_epoch=total_train // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=total_validation // batch_size)

Epoch 1/10


c:\Users\alfia\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


364/364 ━━━━━━━━━━━━━━━━━━━━ 866s 2s/step - accuracy: 0.5826 - loss: 0.7078 - val_accuracy: 0.7613 - val_loss: 0.5125
Epoch 2/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 2s 239us/step - accuracy: 0.6875 - loss: 0.5869 - val_accuracy: 1.0000 - val_loss: 0.3569
Epoch 3/10


c:\Users\alfia\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


364/364 ━━━━━━━━━━━━━━━━━━━━ 823s 2s/step - accuracy: 0.7129 - loss: 0.5666 - val_accuracy: 0.8207 - val_loss: 0.4120
Epoch 4/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 2s 91us/step - accuracy: 0.7188 - loss: 0.5671 - val_accuracy: 1.0000 - val_loss: 0.0607
Epoch 5/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 941s 3s/step - accuracy: 0.7486 - loss: 0.5235 - val_accuracy: 0.8515 - val_loss: 0.4087
Epoch 6/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 3s 234us/step - accuracy: 0.6875 - loss: 0.4896 - val_accuracy: 0.0000e+00 - val_loss: 0.8989
Epoch 7/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 3243s 9s/step - accuracy: 0.7748 - loss: 0.4894 - val_accuracy: 0.8668 - val_loss: 0.3312
Epoch 8/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 12s 624us/step - accuracy: 0.8125 - loss: 0.3706 - val_accuracy: 1.0000 - val_loss: 0.5073
Epoch 9/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 2679s 7s/step - accuracy: 0.7960 - loss: 0.4547 - val_accuracy: 0.8856 - val_loss: 0.2876
Epoch 10/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 5s 494us/step - accuracy: 0.8750 - loss: 0.3425 - val_accu

In [8]:
model.save('gender_detection_model.h5')